## Task 6

In [1]:
import re

s1 = re.compile("[aeiouóyęą]")
s2 = re.compile("i[aeouęą]")

def syllabicator(word):
    if word in ".,;:'\"-=!?()\\":
        return 0
    
    return len(s1.findall(word)) -len(s2.findall(word))
            
               
               
print(syllabicator("kierunek"))
print(syllabicator("konstantynopolitańczykowianeczka"))



3
12


In [2]:

good_accents = set()

for line in open("../resources/pan-tadeusz.txt"):
    line = re.sub('([-.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    
    accents = []
    
    for word in line.lower().split():
        syllables = syllabicator(word)
        if syllables > 0:
            accents.append(syllables)
    if(len(accents)>6): #dlugie sa brzydkie
        continue
    
    if sum(accents) == 13:
        good_accents.add(tuple(accents))
        



In [3]:

suffix_ = re.compile("[^aeiouóyęą]*[aeiouóyęą][^aeiouóyęą]*[aeiouóyęą]")
def get_suffix(word):
    match = suffix_.search(word[::-1])
    if match:
        return match.group(0)[::-1]
    else:
        return word

print(get_suffix("konstantynopolitańczykowianeczka"))
print(get_suffix("pplklpklkjplkj"))

eczka
pplklpklkjplkj


In [4]:
rhymes = {} #from suffix to list of words
bigrams = {} #from snd in bigrams to list of n_syllables to words
unigrams = [{} for _ in range(9)]

for line in open("../poleval_corpus/poleval_2grams.txt"):
    count, prev, word = line.split()
    count = int(count)
    
    word_syllables = syllabicator(word)
    prev_syllables = syllabicator(prev)
    
    if word_syllables <= 0 or prev_syllables <= 0 or \
       word_syllables >= 9 or prev_syllables >= 9:
        continue
    
    suffix = get_suffix(word)
    
    if suffix not in rhymes:
        rhymes[suffix] = [set() for _ in range(9)]
    
    rhymes[suffix][word_syllables].add(word)
    
    unigrams[word_syllables][word] = unigrams[word_syllables].get(word, 0) + count
    unigrams[prev_syllables][prev] = unigrams[prev_syllables].get(prev, 0) + count
    
    if word not in bigrams:
        bigrams[word] = [{} for _ in range(9)]
        
    bigrams[word][prev_syllables][prev] = count
    


In [5]:
for word in bigrams:
    bigrams[word][0]["#SUM#"] = sum([sum(x.values()) for x in bigrams[word]])

unigrams[0]["#SUM#"] = sum([sum(x.values()) for x in unigrams])

In [6]:
print(bigrams["identyczny"][0]["#SUM#"])

1604


In [7]:
#wylosuj rozkład rymów dla dwóch linijek

# wylosuj ostatnie slowo
# wylosuj do niego rym

# losuj słowa

import random

good_accents = list(good_accents)


def make_line(accents, prev):
    
    res = [prev]
    for accent in reversed(accents):
        if prev in bigrams and bigrams[prev][accent]:
            word = random.choices(list(bigrams[prev][accent].keys()), weights=list(bigrams[prev][accent].values()))[0]
        else:
            word = random.choices(list(unigrams[accent].keys()), weights=list(unigrams[accent].values()))[0]
        res.append(word)
        prev = word
    return " ".join(reversed(res))

def get_poem():
    
    good = False
    while not good:
        good = True
        
        second_line_accents = random.choice(good_accents)
        first_line_accents = random.choice(good_accents)
        
        #wylosuj pierwsze słowo
        
        n_syllables = -1
        while n_syllables != first_line_accents[-1]:
            first_word = random.choice(list(bigrams.keys()))
            n_syllables = syllabicator(first_word)
        
        first_suffix = get_suffix(first_word)
        
        if not rhymes[first_suffix][n_syllables]:
            good = False
            continue
            
        second_word = random.choice(list(rhymes[first_suffix][n_syllables]))
        
        first_line = make_line(first_line_accents[:-1], first_word)
        second_line = make_line(second_line_accents[:-1], second_word)
        
    return [first_line, second_line]


get_poem()

['friedl i zmieniono nazwę aleja polishield',
 'tego kutra mięso i gospodarki coalfield']

## Task 7

In [8]:
import math
def ppmi(a, b):
    syll_a = syllabicator(a)
    syll_b = syllabicator(b)
    # bigram (w1,w2) koduje w bigrams[w2][syll_w2][w1]
    # unigram w1 koduje w unigrams[syll_w2][w2]
    
    if b not in bigrams:
        return 0
    if a not in bigrams[b][syll_a]:
        return 0
    
    return max(0, math.log2( bigrams[b][syll_a][a]*unigrams[0]["#SUM#"]/(bigrams[b][0]["#SUM#"]*unigrams[syll_a][a])))
    

print(ppmi("dobry", "dzien"))
print(ppmi("dzien", "dobry"))



5.933089494947373
9.796755318167866


In [9]:


def best_collocation():
    a, b = get_poem()
    print("\n".join([a,b]))
    a = a.split()
    b = b.split()
    print("Collocations:")
    print([ppmi(a[i], a[i+1]) for i in range(len(a)-1)] + [ppmi(a[-1], b[0])])
    print([ppmi(b[i], b[i+1]) for i in range(len(b)-1)] )
    
    # losuj słowa tak, zeby miec lepsza kolokacyjnosc z juz wylosowanym
    for i in range(len(a)-1, 0, -1):
        prev = a[i]
        accent = syllabicator(a[i])
        if prev in bigrams and bigrams[prev][accent]:
            words = random.choices(list(bigrams[prev][accent].keys()), weights=list(bigrams[prev][accent].values()), k=10)
        else:
            words = random.choices(list(unigrams[accent].keys()), weights=list(unigrams[accent].values()), k=10)
        _, best = sorted([(ppmi(word, a[i]), word) for word in words], reverse=True)[0]
        a[i-1] = best
        
    for i in range(len(b)-1, 0, -1):
        prev = b[i]
        accent = syllabicator(b[i])
        if prev in bigrams and bigrams[prev][accent]:
            words = random.choices(list(bigrams[prev][accent].keys()), weights=list(bigrams[prev][accent].values()), k=10)
        else:
            words = random.choices(list(unigrams[accent].keys()), weights=list(unigrams[accent].values()), k=10)
        _, best = sorted([(ppmi(word, b[i]), word) for word in words], reverse=True)[0]
        b[i-1] = best
    
    
    print(" ".join(a))
    print(" ".join(b))
    
    print("Collocations:")
    print([ppmi(a[i], a[i+1]) for i in range(len(a)-1)] + [ppmi(a[-1], b[0])])
    print([ppmi(b[i], b[i+1]) for i in range(len(b)-1)] )   
    
    a = " ".join(a)
    b = " ".join(b)
    
    return "\n ".join([a,b])
    

best_collocation()


broniewskiego już do szyn celem podciętey
iż działalność zostanie sąd biskupi moloney
Collocations:
[1.5766068496720589, 0, 0.08535316344099565, 2.815096780303894, 0, 0]
[2.5300284362148586, 0, 0, 7.618974826028563, 14.160613292044033]
minimalna całkowita likwidacja delegatur terenowych podciętey
zaspokoisz prosiecznego wzniesieniami bogatymi złoceniami moloney
Collocations:
[10.476148512588638, 10.179133418186401, 11.056013430939057, 13.36192801171985, 0, 0]
[0, 15.478844910527535, 11.256237818401184, 14.621768710061723, 0]


'minimalna całkowita likwidacja delegatur terenowych podciętey\n zaspokoisz prosiecznego wzniesieniami bogatymi złoceniami moloney'